# `CDC Process`

## Enable CDC on the Entire Database

In [ ]:
EXEC sys.sp_cdc_enable_db

## Each table needs to be updated or added to system CDC database

In [ ]:
EXEC sys.sp_cdc_enable_table  -- enable CDC for a table. The Language table in this case
@source_schema = N'dbo', 
@source_name   = N'Language', 
@role_name     = NULL

## Shows which databases have CDC enabled

- If CDC is enabled on the database. 1 = yes

In [ ]:
SELECT [name], database_id, is_cdc_enabled  
FROM sys.databases

## Shows the tables enabled for CDC - 1 indicates a CDC table

In [ ]:
SELECT [name], is_tracked_by_cdc   -- lists the tables which are enabled by CDC
FROM sys.tables 
order by name

## Scan of all the CDC Changes

In [ ]:
SELECT * from sys.dm_cdc_log_scan_sessions where empty_scan_count <> 0 


## Show all of the Changes for a CDC Enabled Table

- This is the SQL the Boomi operation will need to run for each table

In [ ]:
SELECT   -- shows the changes for a CDC table, in this case the Area table      
      [__$start_lsn],
      [__$operation]      
      ,[AreaNo]
      ,[PlantNo]
      ,[AreaName]     
  FROM [cdc].[dbo_Area_CT] 
-- from this result, the following is checked on the _$operation column of the returned data set
-- =========
-- 1 = delete
-- 2 = insert
-- 3 = update (captured column values are those before the update operation). 
-- 4 = update (captured column values are those after the update operation)
-- For example:
--  __$operation	    AreaNo	PlantNo	AreaName
--  3	                218	    40	    test area
--  4	                218	    40	    test areas
-- =========

## Show the current retention policy of the CDC database

In [ ]:
select * from cdc.cdc_jobs

## Changes the retention time, in minutes, of the CDC database

In [ ]:
EXECUTE sys.sp_cdc_change_job
   @job_type = N'cleanup',
   @retention = 14400 -- the retention time in minutes
   

## Get the TIme From LSN
the \_$start\_lsn column is in a binary data format and is the time the change was made and recorded in the CDC table. This will convert the binary data to an epoch time

In [ ]:
-- get the time from the LSN
SELECT sys.fn_cdc_map_lsn_to_time(0x00000A1E000042400007)  

## Create View for CDC table which refactors the lsn time into a datatime format

In [ ]:
create view [cdc].[dba_Language_CT_View] as SELECT 
sys.fn_cdc_map_lsn_to_time(__$start_lsn) AS 'rec_ts', * from [cdc].[dbo_Area_CT];

## Remove CDC Process from Database

In [ ]:
EXEC sys.sp_cdc_disable_db  
GO  

## Create an Information View Table for Detailed CDC information

In [ ]:
Create view cdc.Table_Primary_Key_View as select 
lower(b.name) source_table_name,
c.table_catalog source_database,
lower(e.column_name) primary_key,
lower(c.table_catalog + '_cdc_'+ a.capture_instance + '_ct_view') cdc_view_name
from cdc.change_tables a
inner join sys.all_objects b on a.source_object_id = b.object_id
inner join information_schema.tables c on c.table_name = b.name
inner join information_schema.TABLE_CONSTRAINTS d on d.table_name = b.name 
and d.table_schema = c.table_schema
inner join information_schema.CONSTRAINT_COLUMN_USAGE e 
on e.CONSTRAINT_NAME = d.CONSTRAINT_NAME and d.CONSTRAINT_TYPE = 'PRIMARY KEY'; 